### curriculum - 【AIコース】画像認識プロジェクト2

In [ ]:
'''
import cv2
import numpy as np

path = "/Users/yasu/school/curriculum/2018_069/output2/yasu_face/"
files = cv2.imread(path + "0.jpg", 0)

files.shape

In [ ]:
'''
import cv2
import numpy as np
import glob

def make_data():
    path = "/Users/yasu/school/curriculum/2018_069/output2/yasu_face"
    files = glob.glob(path + "/*.jpg")
    transformed_images = np.empty((0, 1296), int)
    for path in files:
        image = cv2.imread(path, 0)
        one_dementional_image = list(image.flatten())
        transformed_images = np.append(transformed_images, 
                                       [one_dementional_image],
                                       axis = 0)
    return(transformed_images)

In [ ]:
'''
data = make_data()
data.shape

In [ ]:
'''
np.savetxt("./output/yasu_face2.csv", data, delimiter = ",")

#### 小課題

##### Model

In [ ]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

In [ ]:
data =  np.loadtxt("./output/yasu_face.csv",  delimiter = ",")
data2 = np.loadtxt("./output/yasu_face2.csv", delimiter = ",")
print("data:", data.shape, " / data2:", data2.shape)

In [ ]:
target = np.ones(data.shape[0]) * 0
target2 = np.ones(data2.shape[0]) * 1
print("target:", target.shape, " / target2:", target2.shape)

In [ ]:
all_train = np.concatenate((data, data2), 0)
all_target = np.concatenate((target, target2), 0)

X_train, X_test, y_train, y_test = train_test_split(all_train, all_target, train_size = 0.9, random_state = 1)

img_rows = 36
img_cols = 36
num_classes = len(set(y_train))
num_classes

In [ ]:
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = X_train.shape[1:4]

X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
#C Build model
model = Sequential()

model.add(Conv2D(1, kernel_size = (3, 3),
                 strides = (1, 1),
                 padding = "same",
                 activation = "relu",
                 bias = True,
                 input_shape = input_shape))

model.add(MaxPooling2D(pool_size = (2, 2),
                       strides = None,
                       padding = "valid"))

model.add(Conv2D(1, kernel_size = (3, 3),
                 strides = (1, 1),
                 padding = "same",
                 bias = True,
                 activation = "relu"))

model.add(MaxPooling2D(pool_size = (2, 2),
                       strides = None,
                       padding = "valid"))

model.add(Flatten())
model.add(Dense(128, activation = "relu"))
# model.add(Dense(num_classes, activation = "softmax"))
model.add(Dense(num_classes, activation = "softmax"))

model.compile(loss = keras.losses.categorical_crossentropy,
              optimizer = keras.optimizers.Adadelta())

model.fit(X_train, y_train, 
          batch_size = 128, 
          epochs = 20, 
          verbose = 1,
          validation_data = (X_test, y_test))

score = model.evaluate(X_test, y_test, verbose = 1)

json_string = model.to_json()
open("face_model.json", "w").write(json_string)
model.save_weights("face_model.h5")

##### 実行

In [ ]:
import cv2
from keras.models import model_from_json

def predict(image):
    model = model_from_json(open("face_model.json").read())
    model.load_weights("face_model.h5")
    result = model.predict_classes(image)
    
    if result == 0:
        return "yasu 1"
    elif result == 1:
        return "yasu 2"
    # elif result == 2:
    #     return ("yasu")

def image_processing(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_gray = cv2.resize(gray, (36, 36))
    image_gray = image_gray.reshape(1, 36, 36, 1)
    image_gray = image_gray.astype("float32")
    image_gray /= 255
    
    return image_gray
    
cascade_path = "/usr/local/opt/opencv/share/OpenCv/haarcascades/haarcascade_frontalface_alt.xml"
cap = cv2.VideoCapture(0)

while True:
    r, img = cap.read()
    
    if r == False:
        continue
    
    cv2.imshow("Capture", img)
    # if cv2.waitKey(1) == 5:
    #C Camera does not stop above condition but it works below condition
    #C Thanks to https://gist.github.com/kurozumi/04a75695dc32c46586be0d69e6a8243f
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    
    cascade = cv2.CascadeClassifier(cascade_path)
    frame_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    facerect = cascade.detectMultiScale(frame_gray, scaleFactor = 1.2, minNeighbors = 3, minSize = (10, 10))
    
    # cv2.namedWindow("a")
    # cv2.namedWindow("b")
    
    if len(facerect) > 0:
        print("face detected", facerect)
        color = (255, 255, 255)
        for rect in facerect:
            x, y = rect[0: 2]
            width, height = rect[2: 4]
            image = img[y - 10: y + height, x: x + width]
            image_processed = image_processing(image)
            result = predict(image_processed)
            print(result)
            
        # cv2.imshow("a", img)
        # cv2.imshow("b", img)
    
    # k = cv2.waitKey(100)
    # if k == 20:
    #     break

#C This enables x button on right top of the capture window
#C Thanks to http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_gui/py_image_display/py_image_display.html
cv2.waitKey(0)  #C This enables x button on right top of the capture window
cap.release()
cv2.destroyAllWindows()
# cv2.waitKey(1)